In [613]:
from yahoofinancials import YahooFinancials
import numpy as np

In [739]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def get_BV_GR_mean(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.mean(gr)
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        if not()
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print('suggested price:', suggestPrice)
        print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'no recommendation due to error'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [715]:
class batch_process:
    def __init__(self, tickers):
        self.tickers = tickers
    def batch_run(self):
        for i in range(np.size(self.tickers)):
            print(self.tickers[i])
            x = cookFinancials(self.tickers[i])
            bv = x.get_BV(20)
            bv.insert(0, x.get_book_value())
            #print(bv)
            bvgr = x.get_BV_GR_median(bv)
            print(bvgr)
            growth = bvgr[1]
            cEPS = x.get_earnings_per_share()
            years = 3;
            rRate = 0.2;
            safty = 0.5
            PE = x.get_PE()
            price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
            #print(price)
            print(x.get_decision(price[1],x.get_current_price()))
            print('=====================================')

In [740]:
x = cookFinancials('SGEN')

In [735]:
bv = x.get_BV(20)
bv

[1876287000, 1273943000, 677569000, 634087000]

In [736]:
bv.insert(0, x.get_book_value())
print(bv)

[2258030000, 1876287000, 1273943000, 677569000, 634087000]


In [737]:
bvgr = x.get_BV_GR_median(bv)
print(bvgr)

(4, 0.3381376283195572)


In [741]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 3;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

66.858
None


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [727]:
print(x.get_earnings_per_share())


0.1640000152780218


In [728]:
x.get_pe_ratio()
x.get_PE()

117.171252

In [729]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

0.1640000152780218
-0.020608307411777323
3
0.2
117.171252
0.5


In [730]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(10.446972911318994, 5.223486455659497)


In [731]:
x.get_decision(price[1],x.get_current_price())

suggested price: 5.223486455659497
stock price: 34.35


'do not buy'

In [628]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [633]:
tickers = gt.get_tickers(AMEX=False)



['DDD', 'MMM', 'WBAI', 'EGHT', 'AHC']

In [ ]:
tickers[:5]

In [642]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=10e3, sectors=gt.SectorConstants.HEALTH_CARE)

In [658]:
print(filtered_by_sector)

['MMM', 'ABT', 'ABBV', 'ALC', 'ABC', 'ANTM', 'BAX', 'BDX', 'BSX', 'BMY', 'CAH', 'CTLT', 'CNC', 'CRL', 'CI', 'CVS', 'DVA', 'RDY', 'EW', 'ELAN', 'LLY', 'FMS', 'GSK', 'HCA', 'HUM', 'IQV', 'JNJ', 'LH', 'MCK', 'MDT', 'MRK', 'MOH', 'NVS', 'NVO', 'OSH', 'PFE', 'QGEN', 'DGX', 'RMD', 'SNN', 'STE', 'SYK', 'TAK', 'TDOC', 'TFX', 'TEVA', 'COO', 'UNH', 'UHS', 'VAR', 'ZBH', 'ZTS', 'ABMD', 'ALXN', 'ALGN', 'ALNY', 'AMGN', 'ARGX', 'AZN', 'BGNE', 'BIIB', 'BMRN', 'BNTX', 'TECH', 'CVAC', 'XRAY', 'DXCM', 'EXAS', 'GMAB', 'GILD', 'GRFS', 'GH', 'HOLX', 'HZNP', 'ICLR', 'IDXX', 'INCY', 'PODD', 'ISRG', 'MASI', 'MRTX', 'MRNA', 'NVCR', 'PPD', 'QGEN', 'REGN', 'RGEN', 'RPRX', 'SNY', 'SRPT', 'SGEN', 'VRTX', 'VTRS', 'WBA']


In [708]:
filtered_by_sector.index('PPD')

83

In [732]:
y = batch_process(filtered_by_sector[83:])

In [733]:
y.batch_run()


PPD
(3, -0.020608307411777323)
suggested price: 5.223486455659497
stock price: 34.35
do not buy
QGEN
(4, 0.005814627707008394)
suggested price: 9.465762384288192
stock price: 47.74
do not buy
REGN
(4, 0.32363436008384605)
suggested price: 295.09961549527037
stock price: 505.13
do not buy
RGEN
(4, 0.3915201932569148)
suggested price: 109.79190250352748
stock price: 187
do not buy
RPRX
(2, 0.04474716423027339)
suggested price: A
stock price: 41.78
no recommendation due to error
SNY
(4, 0.011440178129996492)
suggested price: 11.594338060643393
stock price: 50.62
do not buy
SRPT
(4, 0.19912381391700806)
suggested price: A
stock price: 149.97
no recommendation due to error
SGEN
(4, 0.3381376283195572)


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'